In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v17 (THE EQUALIZER LOGIC)           ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 40.00        
MIN_EDGE = 0.02           
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.10      
KELLY_FRACTION = 0.20     

# --- FIXTURES ---
fixtures = [
    {"Home": "Burnley",           "Away": "Newcastle United",      "Odds_1": 5.29, "Odds_X": 4.14, "Odds_2": 1.62},
    {"Home": "Chelsea",           "Away": "Bournemouth",           "Odds_1": 1.57, "Odds_X": 4.28, "Odds_2": 5.59},
    {"Home": "Nottingham Forest", "Away": "Everton",               "Odds_1": 2.14, "Odds_X": 3.28, "Odds_2": 3.62},
    {"Home": "West Ham United",   "Away": "Brighton",              "Odds_1": 3.15, "Odds_X": 3.54, "Odds_2": 2.24},
    {"Home": "Arsenal",           "Away": "Aston Villa",           "Odds_1": 1.50, "Odds_X": 4.33, "Odds_2": 6.79},
    {"Home": "Manchester United", "Away": "Wolverhampton Wanderers", "Odds_1": 1.35, "Odds_X": 5.35, "Odds_2": 8.47},
    {"Home": "Crystal Palace",    "Away": "Fulham",                "Odds_1": 2.18, "Odds_X": 3.36, "Odds_2": 3.41},
    {"Home": "Liverpool",         "Away": "Leeds United",          "Odds_1": 1.53, "Odds_X": 4.42, "Odds_2": 6.00},
    {"Home": "Brentford",         "Away": "Tottenham Hotspur",     "Odds_1": 2.31, "Odds_X": 3.54, "Odds_2": 3.00},
    {"Home": "Sunderland",        "Away": "Manchester City",       "Odds_1": 7.01, "Odds_X": 4.69, "Odds_2": 1.45},
]

# ==============================================================================
# 2. LOAD MODEL
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded.")
except:
    print("❌ Error loading pickle.")
    exit()

def get_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    stats = {}
    # Auto-map features
    for col in features:
        if 'roll_' in col:
            key = col.replace('home_roll_', '').replace('away_roll_', '')
            if key not in stats: # Avoid overwriting
                raw_col = f"{prefix}roll_{key}"
                stats[key] = last[raw_col] if raw_col in last else 0
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    # --- INPUT ---
    input_data = {
        'diff_elo': (h_elo + 70) - a_elo, 
        'home_elo': h_elo, 'away_elo': a_elo, 'diff_rest': 0, 
    }
    
    # Fill rolling data dynamically
    for feat in features:
        if 'home_roll_' in feat:
            key = feat.replace('home_roll_', '')
            input_data[feat] = h_stats.get(key, 0)
        elif 'away_roll_' in feat:
            key = feat.replace('away_roll_', '')
            input_data[feat] = a_stats.get(key, 0)
        elif 'diff_' in feat and feat not in ['diff_elo', 'diff_rest']:
            key = feat.replace('diff_', '')
            input_data[feat] = h_stats.get(key, 0) - a_stats.get(key, 0)

    # Predict
    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # --- 🧠 THE EQUALIZER LOGIC (MATH FIX) 🧠 ---
    
    # 1. Calculate the Gap
    gap = abs(p_home - p_away)
    
    # 2. Calculate Theoretical Draw Probability
    # If gap is 0, Draw should be ~34%. If gap is big, Draw drops.
    theory_draw = 0.34 - (gap * 0.5)
    
    # 3. Apply Correction
    # If the model is underestimating the draw in a tight game, boost it.
    # is_adjusted = False
    # if p_draw < theory_draw:
    #     diff = theory_draw - p_draw
    #     p_draw = theory_draw
    #     # Steal probability from Home/Away proportionally
    #     p_home -= (diff * 0.5)
    #     p_away -= (diff * 0.5)
    #     is_adjusted = True

    # --- GOLIATH FILTER ---
    block_underdog = False
    if (a_team in ELITE_TEAMS and h_elo < a_elo - 200) or (h_team in ELITE_TEAMS and a_elo < h_elo - 200):
        block_underdog = True


    # --- VALUE FINDER ---
    bets_for_match = []
    
    for outcome, p, odds, label in [
        ('HOME', p_home, f['Odds_1'], h_team),
        ('DRAW', p_draw, f['Odds_X'], 'Draw'),
        ('AWAY', p_away, f['Odds_2'], a_team)
    ]:
        if odds > MAX_ODDS_CAP: continue
        
        # Goliath Safety
        if block_underdog:
            is_favorite = (p == max(p_home, p_draw, p_away))
            if not is_favorite: continue
            
        edge = (p * odds) - 1
        
        # Lower bar for Draws now that probability is corrected
        min_p = 0.23 if outcome == 'DRAW' else 0.20
        
        if edge > MIN_EDGE and p > min_p:
            bets_for_match.append({
                'Match': f"{h_team} vs {a_team}",
                'Type': outcome,
                'Team': label,
                'Odds': odds,
                'Prob': p,
                'Edge': edge,
            })

    # --- SELECTION ---
    if bets_for_match:
        # Sort by Edge
        best = sorted(bets_for_match, key=lambda x: x['Edge'], reverse=True)[0]
        
        # FINAL LOGIC CHECK:
        # If we Adjusted the Draw (Tight Game), and the Draw is Profitable...
        # We prefer it unless the Winner Edge is SIGNIFICANTLY higher.
        # if is_adjusted:
        #     draw_bet = next((b for b in bets_for_match if b['Type'] == 'DRAW'), None)
        #     if draw_bet:
        #         # If Winner Edge isn't at least 10% better than Draw Edge, take the safer Draw
        #         if best['Edge'] < (draw_bet['Edge'] + 0.10):
        #             best = draw_bet
        #             best['Reason'] = "Implied Draw (Tight Game)"
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Reason', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Reason', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 FINAL QUANT SLIP v17 (Equalizer) | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 QUANT ENGINE v17 (THE EQUALIZER LOGIC)           
✅ Model Loaded.

🔍 ANALYZING 10 MATCHES...


NameError: name 'is_adjusted' is not defined